In [1]:
import csv
import os
import glob
import time
import random
from multiprocessing import Pool
import multiprocessing
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
from scipy.interpolate import interp1d
pd.set_option('display.float_format', lambda x: '%.3f' % x)
from os import listdir
from os.path import isfile, join
import pickle as pkl
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.decomposition import PCA,TruncatedSVD,NMF
from dask.delayed import delayed
from nltk.tag.stanford import StanfordNERTagger
#st = StanfordNERTagger('unigrams/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz','unigrams/stanford-ner-2018-02-27/stanford-ner-3.9.1.jar',encoding='utf-8')

In [13]:
pklfiles = [f for f in listdir("/data/dharp/compounding/pkl_datasets/") if isfile(join("/data/dharp/compounding/pkl_datasets/", f))]
len(pklfiles)
done_files=[]
for pklfile in pklfiles:
    done_files.append("/data/dharp/compounding/pkl_datasets/"+pklfile)

In [5]:
def decader(x):
    return(x -x%10)

In [6]:
done_list=[]
for pklfile in pklfiles:
    done_list.append(pklfile.split(".")[0])

In [7]:
br_to_us=pd.read_excel("Book.xlsx")
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))

contextwords_df=pd.read_csv("contexts.csv",sep="\t")
contextwords=contextwords_df.context.tolist()
contextwords_non_pos=set(contextwords_df.context.str.split("_",1).str[0].tolist())
#nouns_df=contextwords_df[contextwords_df.context.str.contains("^.*_n$")]
#nouns=nouns_df.context.tolist()

adv_dict=dict(zip(['adv'],['r']))
adv_replacement={'r1_pos':adv_dict,'r2_pos':adv_dict,'r3_pos':adv_dict,'l1_pos':adv_dict,'l2_pos':adv_dict,'l3_pos':adv_dict}
spelling_replacement={'r1':br_to_us_dict,'r2':br_to_us_dict,'r3':br_to_us_dict,'l1':br_to_us_dict,'l2':br_to_us_dict,'l3':br_to_us_dict,'mod':br_to_us_dict,'head':br_to_us_dict,'word':br_to_us_dict}
decades=[2000, 1990, 1980, 1970, 1960, 1950, 1940, 1930, 1920, 1900, 1910,
            1890, 1880, 1870, 1850, 1860, 1840, 1830, 1820, 1810, 1800]
fivegram_list='a_ aa ab ac ad ae af ag ah ai aj ak al am an ao ap aq ar as at au av aw ax ay az b_ ba bb bc bd be bf bg bh bi bj bk bl bm bn bo bp bq br bs bt bu bv bw bx by bz c_ ca cb cc cd ce cf cg ch ci cj ck cl cm cn co cp cq cr cs ct cu cv cw cx cy cz d_ da db dc dd de df dg dh di dj dk dl dm dn do dp dq dr ds dt du dv dw dx dy dz e_ ea eb ec ed ee ef eg eh ei ej ek el em en eo ep eq er es et eu ev ew ex ey ez f_ fa fb fc fd fe ff fg fh fi fj fk fl fm fn fo fp fq fr fs ft fu fv fw fx fy fz g_ ga gb gc gd ge gf gg gh gi gj gk gl gm gn go gp gq gr gs gt gu gv gw gx gy gz h_ ha hb hc hd he hf hg hh hi hj hk hl hm hn ho hp hq hr hs ht hu hv hw hx hy hz i_ ia ib ic id ie if ig ih ii ij ik il im in io ip iq ir is it iu iv iw ix iy iz j_ ja jb jc jd je jf jg jh ji jj jk jl jm jn jo jp jq jr js jt ju jv jw jx jy jz k_ ka kb kc kd ke kf kg kh ki kj kk kl km kn ko kp kq kr ks kt ku kv kw kx ky kz l_ la lb lc ld le lf lg lh li lj lk ll lm ln lo lp lq lr ls lt lu lv lw lx ly lz m_ ma mb mc md me mf mg mh mi mj mk ml mm mn mo mp mq mr ms mt mu mv mw mx my mz n_ na nb nc nd ne nf ng nh ni nj nk nl nm nn no np nq nr ns nt nu nv nw nx ny nz o_ oa ob oc od oe of og oh oi oj ok ol om on oo op oq or os ot other ou ov ow ox oy oz p_ pa pb pc pd pe pf pg ph pi pj pk pl pm pn po pp pq pr ps pt pu punctuation pv pw px py pz q_ qa qb qc qd qe qf qg qh qi qj ql qm qn qo qp qq qr qs qt qu qv qw qx qy qz r_ ra rb rc rd re rf rg rh ri rj rk rl rm rn ro rp rq rr rs rt ru rv rw rx ry rz s_ sa sb sc sd se sf sg sh si sj sk sl sm sn so sp sq sr ss st su sv sw sx sy sz t_ ta tb tc td te tf tg th ti tj tk tl tm tn to tp tq tr ts tt tu tv tw tx ty tz u_ ua ub uc ud ue uf ug uh ui uj uk ul um un uo up uq ur us ut uu uv uw ux uy uz v_ va vb vc vd ve vf vg vh vi vj vk vl vm vn vo vp vq vr vs vt vu vv vw vx vy vz w_ wa wb wc wd we wf wg wh wi wj wk wl wm wn wo wp wq wr ws wt wu wv ww wx wy wz x_ xa xb xc xd xe xf xg xh xi xj xk xl xm xn xo xp xq xr xs xt xu xv xw xx xy xz y_ ya yb yc yd ye yf yg yh yi yj yk yl ym yn yo yp yq yr ys yt yu yv yw yx yy yz z_ za zb zc zd ze zf zg zh zi zj zk zl zm zn zo zp zq zr zs zt zu zv zw zx zy zz'
pos_replacement={'pos':dict(zip(["noun","verb","adj"],['n','v','a']))}

#temp_list="j_ ja jb jc jd je jf jg jh ji jj jk jl jm jn jo jp jq jr js jt ju jv jw jx jy jz"
fivegram_list=fivegram_list.split(" ")
np.random.shuffle(fivegram_list)

In [8]:
def chunked_dataset_extracter(df):
    df.columns=['fivegram_pos','decade','count']
    df['decade']=np.vectorize(decader)(df['decade'])
    df=df.loc[df.decade.isin(decades)]
    df=df.groupby(['fivegram_pos','decade'])['count'].sum().to_frame()
    df.reset_index(inplace=True)
    df.fivegram_pos=df.fivegram_pos.str.lower()
    df=df.loc[df.fivegram_pos.str.match("^[a-z-]+_[^_]+\s+[a-z-]+_[^_]+\s+[a-z-]+_[^_]+\s+[a-z-]+_[^_]+\s+[a-z-]+_[^_]+$")]
    #df=df.loc[df.fivegram_pos.str.match("^[a-z-]+_[^_]+ [a-z-]+_[^_]+ [a-z-]+_[^_]+ [a-z-]+_[^_]+ [a-z-]+_[^_]+$")]
    df=df.groupby(['fivegram_pos','decade'])['count'].sum().to_frame()
    df.reset_index(inplace=True)
    df.decade=df.decade.astype("int32")
    return(df)

In [2]:
@delayed
def dataset_downloader(letter):
    CHUNKSIZE = 1_000_000
    print("Started with letters "+letter)
    cur_time=time.time()
    
    df_list=[]
    path_loc="http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-5gram-20120701-"+letter+".gz"
    dfs   = pd.read_csv(path_loc, compression='gzip', header=None, sep="\t", quotechar='"',usecols=[0,1,2],chunksize=CHUNKSIZE)
    print("Done reading")
    for df in dfs:
        df_list.append(chunked_dataset_extracter(df))
    complete_df=pd.concat(df_list)
    print("Done collecting datasets")
    print("Saving as pickle file")
    complete_df.to_pickle("/data/dharp/compounding/pkl_datasets/"+letter+".pkl")
    return(letter)

In [ ]:
paths = ["http://storage.googleapis.com/books/ngrams/books/googlebooks-eng-all-5gram-20120701-"+letter+".gz" for letter in letters]

In [10]:
def dataset_downloader_whole():
    num_cores=60


    pool = Pool(num_cores)
    #print("Started with List "+str(list_num))
    #print(partition)
    pool.imap_unordered(dataset_downloader,leftover_list)        
    pool.close()
    pool.join()

In [11]:
def dataset_combiner():
    df_list=[]
    for done_file in done_files:
        df_list.append(pd.read_pickle(done_file))
    entire_df=pd.concat(df_list)
    return entire_df

In [44]:
dataset_downloader_whole()

Started with letters qd
Started with letters zl
Started with letters punctuation
Started with letters qf
Started with letters sf
Started with letters am
Started with letters iy
Started with letters dl
Started with letters os
Started with letters rc
Started with letters ya
Started with letters nz
Started with letters xm
Started with letters xh
Started with letters jr
Started with letters dw
Started with letters gj
Started with letters ze
Started with letters jl
Started with letters dk
Started with letters er
Started with letters vo
Started with letters bk
Started with letters eu
Started with letters le
Started with letters t_
Started with letters vw
Started with letters nk
Started with letters ew
Started with letters wz
Started with letters kw
Started with letters mh
Started with letters tg
Started with letters fi
Started with letters uf
Started with letters hi
Started with letters jz
Started with letters zz
Started with letters si
Started with letters aq
Started with letters ng
Started

In [14]:
to_save_df=dataset_combiner()

In [17]:
to_save_df.reset_index(inplace=True)

In [20]:
to_save_df.drop(["index"],axis=1,inplace=True)

In [ ]:
to_save_df.to_csv("/data/dharp/compounding/datasets/entire_df.csv",sep="\t")